# Prev

## include

In [96]:
include("../../src/struct_data.jl")
include("../../src/neighbor.jl")
include("../../src/forces/forces.jl")
include("../../src/run_event.jl")

RunFusionAggregates (generic function with 1 method)

## model

In [97]:
@time model = ModelSet(
    TimeModel(
        tₛᵢₘ  = 150000.0,
        dt    = 0.5,
        nₖₙₙ  = 100,
        nₛₐᵥₑ = 50
    ),
    InputModel(
        outer_ratio = 0.8,
        path_input  = "../../data/init/Sphere"
    ),
    OutputModel(
        name_output = "Test_1",
        path_output = ""
    ) 
)

Par1, Par2 = Cubic(0.0055,2.0,4.5), ContractilePar(0.164);

dump(model)

  0.000000 seconds (4 allocations: 144 bytes)
ModelSet
  Time: TimeModel
    tₛᵢₘ: Float64 150000.0
    dt: Float64 0.5
    nₖₙₙ: Int64 100
    nₛₐᵥₑ: Int64 50
  Input: InputModel
    outer_ratio: Float64 0.8
    path_input: String "../../data/init/Sphere"
  Output: OutputModel
    name_output: String "Test_1"
    path_output: String ""


## aggregate

In [98]:
agg = nothing
agg = Aggregate(
    [AggType(
        "HEK_1", 
        InteractionPar(Par1, Par2),
        Float32.(readdlm("init_stable.xyz")[3:end,2:end]) |> cu
    )], 
    [AggLocation("HEK_1",[0 0 0]),],
    model
)

agg = FusionAggregate(
    [AggType("HEK_1", InteractionPar(Par1, Par2),agg.Position)], 
    model
)

Aggregate(AggType[AggType("HEK_1", InteractionPar(Cubic{Float64}(0.0055, 2.0, 4.5), ContractilePar(0.164)), 12.189157f0, Float32[-5.8861017 -4.799863 -8.264273; 1.0154873 -5.753389 -9.426559; … ; 3.3784163 3.3059669 9.768439; 6.8377233 4.1324844 8.079775], CuArray{Float32, 2, CUDA.Mem.DeviceBuffer})], AggIndex([1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  2, 2, 2, 2, 2, 2, 2, 2, 2, 2], ["HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1"  …  "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1"]), Float32[-18.075258 -4.799863 -8.264273; -11.173669 -5.753389 -9.426559; … ; 15.567573 3.3059669 9.768439; 19.02688 4.1324844 8.079775], AggGeometry(Float32[12.189157, 12.189157, 12.189157, 12.189157, 12.189157, 12.189157, 12.189157, 12.189157, 12.189157, 12.189157  …  12.189157, 12.189157, 12.189157, 12.189157, 12.189157, 12.189157, 12.189157, 12.189157, 12.189157, 12.1

## kNN

In [99]:
dist = zeros(size(agg.Simulation.Neighbor.idx_red)) |> cu
threads=(100)

@cuda(
    threads=threads,
    blocks=cld.(size(agg.Position,1),threads),
    dist_kernel!(agg.Simulation.Neighbor.idx_red,agg.Simulation.Neighbor.idx_cont,agg.Simulation.Neighbor.idx_sum,dist,agg.Position,agg.Simulation.Parameter.Force.rₘₐₓ)
)
agg.Simulation.Neighbor.idx_red

1000×5008 CuArray{Int32, 2, CUDA.Mem.DeviceBuffer}:
 109   33  19  128  59   43  18   13  …  4974  5007  4960  4790  5004  4969
  42  210  70   34  10   30  30   36     5004  4966  4907  4781  4898  4753
 110  121  13   35  30   59  59   25     4973  5003  4753  4977  4978  4993
  49  241  62   10  41  111  52   43     5002  4974  4993  4675  4974  4881
  34  112  53   49  51  122  43  112     4998  4999  4943  4910  4972  5005
 205  101  45   41  58   36  13   19  …  4947  4958  5008  4979  5000  4975
 213   47  24   65  34   44   5   45     4978  5000  5000  5002  4907  4968
 129   31  54    5  12   52  12  228     5007  4998  4961  4895  5003  4861
   0  219  18    0   7  130   6   44     4977  4992  4934  4888  4966  4907
   0   37  64    0   4   51  60    0     4958  4950     0     0     0  4899
   ⋮                      ⋮           ⋱                       ⋮        
   0    0   0    0   0    0   0    0        0     0     0     0     0     0
   0    0   0    0   0    0   0    0    

# Model

# Force Example

In [103]:
threads=(16,3)
t_nₖₙₙ = 1
CUDA.@time @cuda(
    threads=threads,
    blocks=(cld.(size(agg.Position,1)+1,threads[1]),1),
    sum_force!(agg.Simulation.Neighbor.idx_red,agg.Simulation.Neighbor.idx_cont,agg.Simulation.Neighbor.idx_sum,agg.Position,agg.Simulation.Force.F,agg.Simulation.Parameter.Force,agg.Simulation.Parameter.Contractile.fₚ,model.Time.dt,t_nₖₙₙ,agg.Simulation.Force.Pol)
)

display(agg.Position)
display(agg.Simulation.Force.Pol)

5008×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 -18.0662  -4.83639   -8.30337
 -11.1579  -5.76953   -9.46928
 -12.7862  -0.85681  -11.0101
 -18.6382  -2.35764   -9.38494
 -16.7061  -2.42484  -10.0792
 -14.5669  -3.58609   -9.3752
 -15.0927  -2.39186  -10.8222
 -12.5598  -3.34598  -10.8949
 -17.6868   2.29098   -8.72435
 -17.6192  -1.23717   -9.92766
   ⋮                 
  16.7491   2.22767    9.45561
  10.1317   1.62661   11.0536
  12.9108   4.27913   10.3208
  13.7966   2.97607   10.246
  15.0328   2.0779    10.0443
  18.2589   2.85604    8.79165
  12.9274   5.14689    8.94455
  15.5993   3.30731    9.83328
  19.038    4.16284    8.13507

5008×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
  0.0308286   0.861817   -0.506281
 -0.085355   -0.460537   -0.883527
 -0.590373   -0.297623    0.750254
 -0.167171   -0.299569    0.939315
 -0.219026    0.396102    0.891701
 -0.49781    -0.277086    0.821832
 -0.956028   -0.28958     0.0464196
  0.386054    0.619752   -0.683279
  0.720216   -0.633592   -0.28258
 -0.515107   -0.501574    0.695045
  ⋮                      
  0.365416    0.428555    0.826324
 -0.500224   -0.348656   -0.7926
 -0.505673   -0.861433   -0.0472
  0.0652442   0.981216   -0.181547
 -0.0973992   0.530405    0.842131
  0.866483   -0.471052    0.165281
  0.986343    0.0638961   0.151805
 -0.454432   -0.791056   -0.40954
 -0.76933     0.34173    -0.539771

  0.000207 seconds (52 CPU allocations: 3.609 KiB)


In [104]:
cpu_pol = Matrix(agg.Simulation.Force.Pol)

5008×3 Matrix{Float32}:
  0.0308286   0.861817   -0.506281
 -0.085355   -0.460537   -0.883527
 -0.590373   -0.297623    0.750254
 -0.167171   -0.299569    0.939315
 -0.219026    0.396102    0.891701
 -0.49781    -0.277086    0.821832
 -0.956028   -0.28958     0.0464196
  0.386054    0.619752   -0.683279
  0.720216   -0.633592   -0.28258
 -0.515107   -0.501574    0.695045
  ⋮                      
  0.365416    0.428555    0.826324
 -0.500224   -0.348656   -0.7926
 -0.505673   -0.861433   -0.0472
  0.0652442   0.981216   -0.181547
 -0.0973992   0.530405    0.842131
  0.866483   -0.471052    0.165281
  0.986343    0.0638961   0.151805
 -0.454432   -0.791056   -0.40954
 -0.76933     0.34173    -0.539771

## CUDA functions (Contractile)

In [101]:
# rand_mat = zeros(model.Time.nₖₙₙ,size(agg.Position,1),2) |> cu

# function multirand!(matrix)
#     i  = (blockIdx().x-1) * blockDim().x + threadIdx().x
#     j  = (blockIdx().y-1) * blockDim().y + threadIdx().y

#     if i <= size(matrix,1) && j <= size(matrix,2)
#         matrix[i,j,1] = 2*rand()-1
#         matrix[i,j,2] = sqrt(1-matrix[i,j,1]^2)*(2*rand()-1)
#         # matrix[i,j,1] = rand()
#         # matrix[i,j,2] = rand()
#     end

#     return nothing
# end

# threads = (16,16)
# CUDA.@time @cuda(
#     threads = threads,
#     blocks  = (cld(size(rand_mat,1)+1,threads[1]),cld(size(rand_mat,2)+1,threads[2])),
#     multirand!(rand_mat)
# )

# rand_mat


In [102]:
# function unary_vector(pol_mat)
#     i  = (blockIdx().x-1) * blockDim().x + threadIdx().x

#     if i <= size(pol_mat,1)

#         phi = 2*pi*(2*rand() - 1)

#         pol_mat[i,3] = 2*rand() - 1
#         pol_mat[i,1] = sqrt(1-pol_mat[i,3]^2)*cos(phi)
#         pol_mat[i,2] = sqrt(1-pol_mat[i,3]^2)*sin(phi)

#     end
#     return nothing
# end

# # Calculating Polarization Vectors
# threads=(16)
# CUDA.@time @cuda(
#     threads=threads,
#     blocks=cld.(size(agg.Simulation.Force.Pol,1),threads),
#     unary_vector(agg.Simulation.Force.Pol)
# )


# pol_mat = Matrix(agg.Simulation.Force.Pol)
# open("unit_vec.xyz", "w") do f
#     write(f, "$(size(pol_mat,1))\n")
#     write(f, "t=0\n")
#     writedlm(f,hcat(repeat([1], size(pol_mat,1)),pol_mat), ' ')
# end